### Load utilities

In [1]:
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

* Initializing: paths... done, bridge... done, plugins... fail                 *


In [2]:
ARGS ={
    gridName = "brom-dose.ugx",
    numRefs = 1,
    endTime = 0.1,
    dt = 0.025,
}



In [3]:
print(ARGS.gridName)

brom-dose.ugx


### Initialize UG4 (for 3D and standard algebra)

In [4]:
InitUG(3, AlgebraType("CPU", 1));

In [5]:
requiredSubsets = {"INNER", "WALL", "IN"}

In [6]:
dom = util.CreateDomain(ARGS.gridName, 0, requiredSubsets)

Loading Domain brom-dose.ugx ... done.
Performing integrity check on domain ... done.


In [7]:
util.refinement.CreateRegularHierarchy(dom, ARGS.numRefs, true)


util.refinement: - refining level 0


### Create Approximation space

In [8]:
approxSpaceDesc = { fct = "c", type = "Lagrange", order = 1 }

In [9]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct(approxSpaceDesc.fct, approxSpaceDesc.type, approxSpaceDesc.order)
approxSpace:init_levels()
approxSpace:init_top_surface()
print("Approximation space:")
approxSpace:print_statistic()

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |     0: INNER |      1: WALL |       2: TOP |        3: IN |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |          104 |            6 |           82 |           13 |            3 |
| (lev,    1)    |          555 |          169 |          343 |           37 |            6 |
| (lev,    0, g) |          104 |            6 |           82 |           13 |            3 |
| (lev,    1, g) |          555 |          169 |          343 |           37 |            6 |
| (surf, top)    |          555 |      

## Create a convection-diffusion-equation

In [10]:
elemDisc = ConvectionDiffusion("c", "INNER", "fv1")
elemDisc:set_diffusion(10)

In [11]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(0.0, "c", "IN")

In [12]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc)
domainDisc:add(dirichletBnd)

## Solver setup

In [13]:
solverDesc = 
{
    type = "bicgstab",
    precond = 
    {
        type		= "gmg",
        approxSpace	= approxSpace,
        smoother	= "ilu",
        baseSolver	= "lu"
    }
}
solver = util.solver.CreateSolver(solverDesc)

## Solve transient problem

In [14]:
function InitialValue(x,y,z,t,si) 
    if (z<1.75) then return 0.0 else return 1.0 end
end

In [15]:
local u = GridFunction(approxSpace)
u:set(0.0)
Interpolate("InitialValue", u, "c")


local startTime = 0.0
util.SolveLinearTimeProblem(u, domainDisc, solver, VTKOutput(), "sol_brom",
                            "ImplEuler", 1, startTime, ARGS.endTime, ARGS.dt); 

SolveLinearTimeProblem, Linear Solver setup:
BiCGStab( restart = 0, min_orthogonality = 0)
 Convergence Check: StdConvCheck( max steps = 100, min defect = 1e-12, relative reduction = 1e-06)
 Preconditioner: 
 | GeometricMultigrid (V-Cycle)
 |  Smoother (3x pre, 3x post): ILU( damping = ConstantDamping(1))
 |  Basesolver ( Baselevel = 0, gathered base = false): AgglomeratingSolver: SuperLU

>> Writing start values
++++++ TIMESTEP 1 BEGIN (current time: 0) ++++++
++++++ Time step size: 0.025
++++++ Assembling Matrix/Rhs for step size 0.025

   % %%%%%%%%             BiCGStab              %%%%%%%%%%%
   % %%%%%%%%   (Precond: Geometric MultiGrid)  %%%%%%%%%%%
   %   Iter      Defect         Rate 
   %    0:    8.211933e-01      -------
   %    1:    2.872137e-04    3.497517e-04
   %    2:    4.098633e-07    1.427032e-03
   % Relative reduction 1.000000e-06 reached after 2 steps.
   % Average reduction over 2 steps: 7.064750e-04
   % %%%%%  Iteration converged  %%%%%

++++++ TIMESTEP 1 END

In [16]:
ls

LUA-ERROR: ---

LUA-ERROR: 
[string "buffer"]:1: '=' expected near '<eof>'
 % ABORTING script parsing.
